## Mask Detection with Deep Learning

This Jupyter Notebook implements a deep learning model for classifying images containing faces with or without masks. It utilizes transfer learning with MobileNetV2 and data augmentation techniques to improve generalization performance.

**1. Imports**

In [ ]:
# Import Dependencies
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

This cell imports necessary libraries for image processing, deep learning model building, data manipulation, and visualization.

**2. Define Paths and Categories**

In [ ]:
DIRECTORY = r"C:\Users\Chamod Peiris\Documents\GitHub\Projects_24\Sys_MaskDetection\data"
CATEGORIES = ["with_mask", "without_mask"]

This cell defines the directory containing the training images and the two categories (with_mask and without_mask). Update the `DIRECTORY` path to point to your actual data location.

**3. Load and Preprocess Images**

In [ ]:
data = []
labels = []

for category in CATEGORIES:
	path = os.path.join(DIRECTORY, category)
	for img in os.listdir(path):
		img_path = os.path.join(path, img)
		image = load_img(img_path, target_size=(224, 224))
		image = img_to_array(image)
		image = preprocess_input(image)

		data.append(image)
		labels.append(category)

This section iterates through each category, loads images, resizes them to a fixed size (224x224 pixels), preprocesses them for the MobileNetV2 model (including normalization), and stores them in the `data` and `labels` lists.

**4. Encode Labels and Split Data**

In [ ]:
#Encode and convert to numpy array
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

(X_train, X_test, y_train, y_test) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=42)

- Label Binarizer is used to convert category labels ("with_mask" and "without_mask") into one-hot encoded vectors.
- `to_categorical` transforms the one-hot encoded vectors into a binary matrix suitable for multi-class classification.
- `train_test_split` splits the data into training and testing sets, maintaining the class distribution using stratification.

**5. Data Augmentation**

In [ ]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

This cell defines an `ImageDataGenerator` object for data augmentation. Here, various transformations like rotation, zoom, shift, shear, and horizontal flip are applied to artificially increase the training data size and improve model robustness.

**6. Define Model Architecture**

In [ ]:
INIT_LR = 1e-4

# MobileNetV2 model without the head
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# Head model on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dense(32, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="sigmoid")(headModel)

# Combine the base and head model
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze head
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=INIT_LR),
	metrics=["accuracy"])

# summarize the model
print(model.summary())

- This section defines the model architecture.
  - A pre-trained MobileNetV2 model is loaded with its top layers excluded (transfer learning).
  - A custom head model is created on top of the pre-trained base, consisting of average pooling, flattening, dense layers with ReLU activation, dropout for regularization, and a final output layer with sigmoid activation for binary classification.
  - The base model layers are frozen to prevent retraining learned features, focusing on adapting the head model for the mask detection task.
  - The model is compiled with binary cross-entropy loss for multi-class classification, Adam optimizer with an initial learning rate, and accuracy metric.
  - Finally, the model summary is printed.

**7. Train the Model**

In [ ]:
EPOCHS = 20
BS = 32

# head model training
H = model.fit(
	aug.flow(X_train, y_train, batch_size=BS),
	steps_per_epoch=len(X_train) // BS,
	validation_data=(X_test, y_test),
	validation_steps=len(y_test) // BS,
	epochs=EPOCHS)

predtest = model.predict(X_test, batch_size=BS)

# index of the label with the corresponding largest predicted probability
predtest = np.argmax(predtest, axis=1)

# classification report
print(classification_report(y_test.argmax(axis=1), predtest,
	target_names=lb.classes_))

This section defines the model training and the predictions;
*   Model trained for 20 EPOCHS which took considerable time but ended up giving proper results.
*   Predictions were quite good even the accuracy remains 0.99 at max.

